# PSyclone tutorial: Use of Fparser2 in PSyclone

This example shows how PSyclone reads existing Fortran code using a parser called fparser2 (https://pypi.org/project/fparser/).

First let's specify a simple Fortran code in a Python string. PSyclone typically reads code from file but it is easier to demonstrate what is happening here using a string.

In [1]:
code = '''program test
  implicit none
  integer, parameter :: ni=10, nj=10, nk=10
  real, dimension(ni,nj,nk) :: a,b
  integer :: i,j,k
  do k=1,nk
    do j=1,nj
      do i=1,ni
        a(i,j,k) = b(i,j,k)
      end do
    end do
  end do
end program test'''

PSyclone parses this code using a Fortran parser called fparser2 (see https://fparser.readthedocs.io/en/stable/ if you are interested). Notice that we choose the Fortran2003 standard (`std=f2003`) below. It is also possible to choose Fortran 2008 (`std=f2008`), however newer standards are not yet available in fparser2.

In [2]:
from fparser.common.readfortran import FortranStringReader
reader = FortranStringReader(code)
from fparser.two.parser import ParserFactory
parser = ParserFactory().create(std="f2003")
parse_tree = parser(reader)

The parsed code (stored in the `parse_tree` variable) can be printed out. As expected, the same code that we defined in the original code is output. The only way to know that something has happened is that the formatting and case has changed.

In [3]:
print(str(parse_tree))

PROGRAM test
  IMPLICIT NONE
  INTEGER, PARAMETER :: ni = 10, nj = 10, nk = 10
  REAL, DIMENSION(ni, nj, nk) :: a, b
  INTEGER :: i, j, k
  DO k = 1, nk
    DO j = 1, nj
      DO i = 1, ni
        a(i, j, k) = b(i, j, k)
      END DO
    END DO
  END DO
END PROGRAM test


The parser has created a tree of nodes which conform to the rules that specify the Fortran 2003 language. Next we print out this tree as text to see all of the nodes that have been created. The names of these nodes correspond to the rules that specify the Fortran 2003 language. If you are interested you can see these rules here (https://wg5-fortran.org/N1601-N1650/N1601.pdf).

In [4]:
print(repr(parse_tree))

Program(Main_Program(Program_Stmt('PROGRAM', Name('test')), Specification_Part(Implicit_Part(Implicit_Stmt('NONE')), Type_Declaration_Stmt(Intrinsic_Type_Spec('INTEGER', None), Attr_Spec_List(',', (Attr_Spec('PARAMETER'),)), Entity_Decl_List(',', (Entity_Decl(Name('ni'), None, None, Initialization('=', Int_Literal_Constant('10', None))), Entity_Decl(Name('nj'), None, None, Initialization('=', Int_Literal_Constant('10', None))), Entity_Decl(Name('nk'), None, None, Initialization('=', Int_Literal_Constant('10', None)))))), Type_Declaration_Stmt(Intrinsic_Type_Spec('REAL', None), Attr_Spec_List(',', (Dimension_Attr_Spec('DIMENSION', Explicit_Shape_Spec_List(',', (Explicit_Shape_Spec(None, Name('ni')), Explicit_Shape_Spec(None, Name('nj')), Explicit_Shape_Spec(None, Name('nk'))))),)), Entity_Decl_List(',', (Entity_Decl(Name('a'), None, None, None), Entity_Decl(Name('b'), None, None, None)))), Type_Declaration_Stmt(Intrinsic_Type_Spec('INTEGER', None), None, Entity_Decl_List(',', (Entity_De

So, why do we use fparser2 in PSyclone? The main reason is that it makes it very easy to extract information about the code in a robust way. Originally the code was stored as a string, now it is stored in a structured way. For example, if we want to get access to all the assignment statements then we can use the `walk` utility (https://fparser.readthedocs.io/en/latest/fparser2.html#fparser.two.utils.walk)...

In [5]:
from fparser.two.utils import walk
from fparser.two import Fortran2003
assignments = walk(parse_tree, Fortran2003.Assignment_Stmt)
assign = assignments[0]
print(assign)

a(i, j, k) = b(i, j, k)


Every node in the fparser2 parse tree has the `parent` and `children` properties to help with tree navigation:

In [6]:
print(assign.children)
print(assign.children[0].parent)

(Part_Ref(Name('a'), Section_Subscript_List(',', (Name('i'), Name('j'), Name('k')))), '=', Part_Ref(Name('b'), Section_Subscript_List(',', (Name('i'), Name('j'), Name('k')))))
a(i, j, k) = b(i, j, k)


It is also possible to modify the parse tree, although it is currently a little cumbersome due to the use of tuples which are immutable. Improving this in fparser2 is work in progress. In the example below we change the values of the `ni`, `nj` and `nk` parameters.

In [7]:
for node in walk(parse_tree, Fortran2003.Int_Literal_Constant):
    children = list(node.items)
    children[0] = "20"
    node.items = tuple(children)
print(parse_tree)

PROGRAM test
  IMPLICIT NONE
  INTEGER, PARAMETER :: ni = 20, nj = 20, nk = 20
  REAL, DIMENSION(ni, nj, nk) :: a, b
  INTEGER :: i, j, k
  DO k = 20, nk
    DO j = 20, nj
      DO i = 20, ni
        a(i, j, k) = b(i, j, k)
      END DO
    END DO
  END DO
END PROGRAM test


If you would like to try something yourself why not try to print out all of the  `do` statements. Hint: see the `repr` output to find the name of the `do` node and then follow the approach taken for printing assignment statements.

Congratulations. You've completed the fparser2 section of the tutorial.
Back to the [Introduction section](../introduction.ipynb).